Learned:
- another appearance of broadcasting
- more numpy methods
- indentation/spacing customization

In [1]:
import numpy as np

WCSS cost function

In [8]:
def WCSS(X, curr_clusters, centroids):
	total_cost = 0
	for i, centroid in enumerate(centroids):
		#slice data set to get data for current cluster
		cluster_data = X[curr_clusters == i]
		#compute normalized distance between datum and centroid,
		distances = np.linalg.norm(cluster_data - centroid, axis=1)**2
		#add to total
		total_cost += np.sum(distances)
	return total_cost

kmeans

In [ ]:
class KMeans:
    def __init__(self, max_epochs=100):
        # Set model parameters
        self.max_epochs = max_epochs
        self.centroids = None
        self.cost_storage = []

    def fit(self, X_train, k):
        # Convert to numpy array, choose k data points to initialize centroids
        X_train = X_train.to_numpy()
        random_indices = np.random.choice(X_train.shape[0], size=k, replace=False)
        self.centroids = X_train[random_indices]

        for epoch in range(self.max_epochs):
            prev_centroids = self.centroids
            # Calculate distances between each point and the centroids
            curr_distances = np.linalg.norm(X_train[:, np.newaxis, :] - self.centroids, axis=2)
            curr_clusters = np.argmin(curr_distances, axis=1)

            # Initialize new centroids array
            new_centroids = np.zeros_like(self.centroids)
            
            # Update centroids using an external for loop
            for i in range(k):
                points_in_cluster = X_train[curr_clusters == i]
                
                if len(points_in_cluster) > 0:  # Avoid empty clusters
                    new_centroids[i] = np.mean(points_in_cluster, axis=0)
                else:
                    new_centroids[i] = self.centroids[i]  # Keep previous centroid if cluster is empty

            self.centroids = new_centroids

            # Compute the current cost (WCSS) and store it
            curr_cost = WCSS(X_train, curr_clusters, self.centroids)
            self.cost_storage.append(curr_cost)

            if np.allclose(self.centroids, prev_centroids, atol=1e-6):
                break

        # Final cluster assignments
        self.clusters = curr_clusters
    
    def predict(self, X_test):
        X_test = X_test.to_numpy()
        distances = np.linalg.norm(X_test[:, np.newaxis] - self.centroids, axis=2)
        clusters = np.argmin(distances, axis=1)
        return clusters
    
    #return centroids
    def get_centroids(self):
        return self.centroids
    
    def get_cov_matrices(self, X_train, k):
        covariances = []
        for i in range(k):
            # Get points assigned to cluster k
            cluster_data = X_train[self.clusters == i]
            # Compute the covariance matrix for the cluster
            if len(cluster_data) > 1:
                cov_matrix = np.cov(cluster_data, rowvar=False)
            else:
                cov_matrix = np.zeros((X_train.shape[1], X_train.shape[1]))  # Handle small cluster cases
            covariances.append(cov_matrix)
        return np.array(covariances)
    
    def get_priors(self, X_train, k):
        priors = []
        for i in range(k):
            # Get points assigned to cluster k
            cluster_data = X_train[self.clusters == i]
            # Compute the class prior
            prior = (cluster_data.shape[0]+1)/(X_train.shape[0]+k)
            priors.append(prior)
        return np.array(priors)
    
    #return clusters
    def get_clusters(self):
        return self.clusters

EMA

In [ ]:
class EMA:
    def __init__(self, max_epochs=100, var_smoothing = 1e-9):
        # Set model parameters
        self.var_smoothing = var_smoothing
        self.max_epochs = max_epochs
        self.means = None
        self.cov_matrices = None
        self.class_priors = None
        self.resp = None
    
    def fit(self, X_train, k):
        # Convert to numpy array, choose k data points to initialize centroids
        X_train = X_train.to_numpy()
        kmeans_approx = KMeans()
        kmeans_approx.fit(X_train, k)
        
        self.means = kmeans_approx.get_centroids()
        self.cov_matrices = kmeans_approx.get_cov_matrices()
        self.class_priors = kmeans_approx.get_priors()
        
        prev_log_likelihood = None
        
        for epoch in range(self.max_epochs):
            # E step
            inv_cov_matrices = np.linalg.inv(self.cov_matrices + self.var_smoothing * np.eye(X_train.shape[1]))

            diff = X_train[:, np.newaxis, :] - self.means
            quadratic_form = np.einsum('ijk,ikl,ijl->ij', diff, inv_cov_matrices, diff)

            # Compute log-likelihoods without constant terms
            log_likelihoods = (-0.5 * quadratic_form) + np.log(self.class_priors)
            
            log_sum_exp = np.log(np.sum(np.exp(log_likelihoods), axis=1, keepdims=True))
            self.resp = np.exp(log_likelihoods - log_sum_exp)
            
            # Calculate the total log-likelihood for this epoch
            total_log_likelihood = np.sum(log_sum_exp)

            # Convergence check
            if prev_log_likelihood is not None and np.allclose(total_log_likelihood, prev_log_likelihood, atol=1e-6):
                print(f"Converged at epoch {epoch}")
                break  # Stop if the total log-likelihood change is below the threshold
    
            prev_log_likelihood = total_log_likelihood
    
            # M-step
            self.update_means(X_train)
            self.update_priors(X_train)
            self.update_cov_matrices(X_train)
            
    def predict(self, X_test):
        X_test = X_test.to_numpy()
        # Compute the inverse of the covariance matrices
        inv_cov_matrices = np.linalg.inv(self.cov_matrices)
        
        # Compute the difference between the test data points and the means of each cluster
        diff = X_test[:, np.newaxis, :] - self.means  # Shape: (n_samples, k, n_features)
        
        # Compute the quadratic form for the Mahalanobis distance
        quadratic_form = np.einsum('ijk,ikl,ijl->ij', diff, inv_cov_matrices, diff)  # Shape: (n_samples, k)
        
        # Compute log-likelihoods without the constant terms
        log_likelihoods = (-0.5 * quadratic_form) + np.log(self.class_priors)
        
        # Assign each sample to the cluster with the highest log-likelihood
        predicted_clusters = np.argmax(log_likelihoods, axis=1)  # Shape: (n_samples,)
        
        return predicted_clusters
            
            
            
    def update_means(self,X_train):
        weighted_sums = np.sum(self.resp[:, :, np.newaxis] * X_train[:, np.newaxis, :], axis=0)
        # Denominator: Sum of the responsibilities for each cluster (to normalize the means)
        responsibility_sums = np.sum(self.resp, axis=0)[:, np.newaxis]
        # Compute the new means
        self.means = weighted_sums / responsibility_sums
    
    def update_priors(self,X_train):
        self.class_priors = np.sum(self.resp, axis=0) / X_train.shape[0]
        
    def update_cov_matrices(self,X_train):
        means = self.means[np.newaxis, :, :]  # Shape: (1, k, n_features)
        # Compute the difference between data points and means for each cluster
        diff = X_train[:, np.newaxis, :] - means  # Shape: (n_samples, k, n_features)
        # Compute the weighted outer product for each data point and cluster
        # The result will be a matrix of shape (n_samples, k, n_features, n_features)
        weighted_outer_products = (self.resp[:, :, np.newaxis, np.newaxis] * 
                                diff[:, :, :, np.newaxis] * diff[:, :, np.newaxis, :])
        # Sum over samples to get the covariance matrix for each cluster
        cov_matrices = np.sum(weighted_outer_products, axis=0) / np.sum(self.resp, axis=0)[:, np.newaxis, np.newaxis]
        self.cov_matrices = cov_matrices